# Scala implicits

Implicits in Scala are also pretty ubiquous. For instance, we can find them in the Scala standard library, and two popular frameworks: akka stream and Spark.

![futureapi](../misc/futureapi.png)

![futureapply](../misc/futureapply.png)

![futurefoldleft](../misc/futurefoldleft.png)

![akka-flow](../misc/akka-api-flow.png)

![akka-flow-runwith](../misc/akka-api-flow-runwith.png)

![spark-dataset](../misc/spark-api-dataset.png)

![dataset-flatmap](../misc/spark-api-dataset-flatmap.png)

![spark-groupbykey](../misc/spark-api-dataset-groupbykey.png)

# Implicits as term inference

We already saw that the Scala compiler can infer the type parameters that have to be applied in a call to a generic function. For instance, given:

In [ ]:
def foo[T](i: T): T = i

we may call our function passing explicitly the type parameter:

In [ ]:
foo[Int](1)

or we may omit the type parameter and let the Scala compiler to infer its proper value:

In [ ]:
foo(1)

Can values also be inferred by the Scala compiler? Yes, they can! For instance, in order to create an asynchronous computation using Scala `Future`s, we need to pass its `ExecutionContext`, i.e. the component that will be in charge of executing it. This is the signature of the factory method: 

![futureapply](../misc/futureapply.png)

As you can see, the execution context parameter is maked as `implicit` which means that the Scala compiler, in principle, can infer the value of that parameter for us. Of coruse, we can also pass the parameter explicitly:

In [ ]:
import scala.concurrent.{Future, ExecutionContext}

Future[Int](1)(ExecutionContext.global)

but the idiomatic way to write this invocation is as follows:

In [ ]:
Future(1)

Ooops. We forgot an important detail. In order for Scala to infer the type parameter we didn't have to do anything: Scala infers the type `Int` just from the type of the code to be executed. However, in order to infer the execution context, Scala needs some help in the form of `implicit` declarations: 

In [ ]:
implicit val ec: ExecutionContext = ExecutionContext.global

Future(1)

In sum, if we want Scala to infer the value of some parameter we mark it as `implicit`. If we want some value to be used as the implicit parameter for a given type, we declare that value as `implicit`.

# Implicits for context parameters

The previous example shows a very common use case of implicit parameters: to inject runtime dependencies, i.e. contextual parameters that are needed in order to execute our code. This is the case with the `ExecutionContext` for `Future`-based computations, or the `Materializer` component that we need to run a streaming pipeline of Akka stream. Let's analyse this later example with some more detail.

In [1]:
import $ivy.`com.typesafe.akka::akka-stream:2.6.4`
import java.time._
import scala.concurrent._, duration._
import akka._
import akka.actor._
import akka.stream._
import akka.stream.scaladsl._

repl.pprinter() = repl.pprinter().copy(defaultHeight = 5 )

import $ivy.$                                     

import java.time._

import scala.concurrent._, duration._

import akka._

import akka.actor._

import akka.stream._

import akka.stream.scaladsl._



Let's say that we have this pipeline that simply generates a stream of integer values which are then printed in the console:

In [6]:
val pipeline: RunnableGraph[NotUsed] = 
    Source(List(1,2,3,4)).to(Sink.foreach(println))

cmd5.sc:8: materializing requested scala.reflect.type.ClassTag[akka.stream.scaladsl.RunnableGraph[akka.NotUsed]] using scala.reflect.`package`.materializeClassTag[akka.stream.scaladsl.RunnableGraph[akka.NotUsed]]()
  .printOnChange(pipeline, "pipeline", _root_.scala.None, _root_.scala.None, _root_.scala.None)) }
                ^

pipeline: RunnableGraph[NotUsed] = RunnableGraph(
  LinearTraversalBuilder(
    None,
    None,
...

This is just a declarative program. If we want our pipeline to be executed we need to `run` it:

In [7]:
pipeline.run

cmd6.sc:8: materializing requested scala.reflect.type.ClassTag[akka.NotUsed] using scala.reflect.`package`.materializeClassTag[akka.NotUsed]()
          .print(res6, "res6", _root_.scala.None)
                ^

res6: NotUsed = NotUsed

Oooops (again). In order to run a pipeline we need a `Materializer`, as the `run` signature shows:

![run](../misc/runnablegraph.png)

![runnablegraphrun](../misc/runnablegraphrun.png)

The standard `Materializer` that is provided by the akka stream library executes pipelines by means of an actor system. So, in order to declare the implicit `Materializer` value, we first need to create an `ActorSystem`:

In [2]:
implicit val system: ActorSystem = ActorSystem("akka-stream-primer")

system: ActorSystem = akka://akka-stream-primer

and we can now run our pipeline: 

In [8]:
pipeline.run

cmd7.sc:8: materializing requested scala.reflect.type.ClassTag[akka.NotUsed] using scala.reflect.`package`.materializeClassTag[akka.NotUsed]()
          .print(res7, "res7", _root_.scala.None)
                ^1
2
3
4


res7: NotUsed = NotUsed

But, wait, Scala needs an implicit value of the type `Materializer`, and we defined an implicit value of type `ActorSystem`. How did Scala infer an implicit value for the former type? The answer lies in the following declaration located in the companion object of the `Materializer` class: 

In [ ]:
/*

object Materializer {

  implicit def matFromSystem(
      implicit provider: ClassicActorSystemProvider): Materializer =
    ???

}
*/

`matFromSystem` is called a _conditional implicit value_, meaning that Scala can construct an implicit value for the `Materializer` type, _provided that_ it can infer an implicit value for a `ClassicActorSystemProvider`. In our example, it certainly can, since we declare an implicit value of the type `ActorSystem` (which extends the trait `ClassicActorSystemProvider`). Ok, but how did the Scala compiler know that there were such a conditional implicit value? Because it was not certainly in scope. Right, but `matFromSystem` is defined in the _companion object_ of the `Materializer` type, and Scala looks automatically for implicits there.

In [23]:
interp.configureCompiler(_.settings.XlogImplicits.value = false)

In [24]:
interp.configureCompiler(_.settings.logImplicitConv.value = false)

In [25]:
interp.configureCompiler(_.settings.warnExtraImplicit.value = false)


# Implicits for class extension

Another common use case for implicits is the implementation of extension methods: there is a class for which we have no control, or no interest in modifying its definition, and, yet, we would like to add some new methods. The Scala standard library of Scala has good examples of this pattern. For instance, the `String` class comes actually from Java:

In [19]:
val s: String = "hi"
s.getClass

s: String = "hi"
res18_1: Class[T] = class java.lang.String

and `java.lang.String` does not declares a `map` method. How then can we make this call?

In [20]:
s.map(c => c.toUpper)

res19: String = "HI"

The answer lies in the following declaration in the `Predef.scala` file:

In [ ]:
/*
  implicit def augmentString(x: String): StringOps = 
      new StringOps(x)
*/

This is not a conditional implicit value, because the argument `x` is not implicit. This is a so-called *implicit conversion* that allows the Scala compiler to convert a `String` into an object of type `StringOps`. This class is the one that actually provides the new method `map`. It looks like this:

In [22]:
class MyStringOps(x: String){
    def mymap(f: Char => Char): String = 
        "dummy"
}

defined class MyStringOps

Now, in order to extend the `String` class with this new method, we have to do something similar to what the Scala library does, i.e. provide an implicit conversion:

In [26]:
implicit def toMyStringOps(x: String): MyStringOps = 
    new MyStringOps(x)

defined function toMyStringOps

And now it works!

In [27]:
"hiii".mymap(identity)

res26: String = "dummy"

This pattern is so common that Scala 2.10 provided a special construct for it: *implicit classes*.

In [32]:
implicit class MyStringOps2(x: String){
    def mymap2(f: Char => Char): String = 
        "dummy"
}

defined class MyStringOps2

In [33]:
"oooo".mymap2(identity)

res32: String = "dummy"

# Implicits for ad-hoc polymorphism

Ordering[T]
Encoder[T]

# Implicits for type-level computing